In [2]:
### This was based on https://github.com/aclarkData/Journal-Entry-Machine-Learning

In [1]:
import pandas as pd

In [4]:
je_data = pd.read_csv("IAS_GL_JOURNAL_DETAIL-2232370.xls")

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
### Convert date object to date
je_data['Date'] = pd.to_datetime(je_data['Date'])

0        2017-01-31
1        2017-01-31
2        2017-01-31
3        2017-01-31
4        2017-01-31
5        2017-01-31
6        2017-01-31
7        2017-01-31
8        2017-01-31
9        2017-01-31
10       2017-01-31
11       2017-01-31
12       2017-01-31
13       2017-01-31
14       2017-01-31
15       2017-01-31
16       2017-01-31
17       2017-01-31
18       2017-01-31
19       2017-01-31
20       2017-01-31
21       2017-01-31
22       2017-01-31
23       2017-01-31
24       2017-01-31
25       2017-01-31
26       2017-01-31
27       2017-01-31
28       2017-01-31
29       2017-01-31
            ...    
709803   2017-01-31
709804   2017-01-31
709805   2017-01-31
709806   2017-01-31
709807   2017-01-31
709808   2017-01-31
709809   2017-01-31
709810   2017-01-31
709811   2017-01-31
709812   2017-01-31
709813   2017-01-31
709814   2017-01-31
709815   2017-01-31
709816   2017-01-31
709817   2017-01-31
709818   2017-01-31
709819   2017-01-31
709820   2017-01-31
709821   2017-01-31


In [9]:
### Convert date to int
je_data['Date Int'] = je_data['Date'].astype(str).astype(int)

ValueError: invalid literal for long() with base 10: '01/31/2017'

In [12]:
je_data.dtypes

Account          object
Date             object
Sum Amount      float64
Line Descr       object
Journal ID       object
User             object
Unit             object
Resource         object
Activity         object
Physical Loc     object
Project          object
Currency         object
dtype: object

In [3]:
from sklearn.feature_extraction import DictVectorizer

# Turn categorical features into 1-hot encoded features
categorical_features = je_data[['Account', 'User', 'Unit', ]]
dv = DictVectorizer()
cat_matrix = dv.fit_transform(categorical_features.T.to_dict().values())